In [7]:
import re
import pandas as pd
import sqlite3
from py2neo import Graph, Node, Relationship
def read_derive_onlyfor_rules():
    query = f"""
    SELECT Component, [Project name], Version, Date, Owner, [OD rules], rules FROM T3_table
    WHERE Type = 'Derive' AND [OD rules] IS NOT NULL AND [OD rules] LIKE '%Only for%' AND [OD rules] NOT LIKE '%✔%' """
    conn = sqlite3.connect('T3.db')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    components = [row[0] for row in data if row[0] is not None]
    project_names = [row[1] for row in data if row[1] is not None]
    versions = [row[2] for row in data if row[2] is not None]
    dates = [row[3] for row in data if row[3] is not None]
    owners = [row[4] for row in data if row[4] is not None]
    OD_rules = [row[5] for row in data if row[5] is not None]
    rules = [row[6] for row in data if row[6] is not None]
    return components, project_names, versions, dates, owners, OD_rules, rules
components, project_name, version, date, owner, OD_rules, rules = read_derive_onlyfor_rules()
newrules = [line.split(']')[0] for line in rules]
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
i = 0
for od in OD_rules:
    index = od.find(':') 
    after = od[index:].strip()
    after = after.replace(':', '') 
    items = re.split(r'/', after) 
    node1 = Node(
        "parent",
        Name = newrules[i],
        Component = components[i],
        originalRule = rules[i],
        Type = "Derive",
        ProjectName = project_name[i],
        Version = version[i],
        owner = owner[i], 
        date = date[i],
        ruleIndex = od)
    g.create(node1) 
    for item in items:
        node2 = Node(
            "son", 
            Name = item,
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = od)
        g.create(node2)
        relationship = Relationship(node2, "should", node1) 
        g.create(relationship) 
    i += 1 